In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import cvxpy as cp

from free_flyer.free_flyer import FreeFlyer
import pickle, os

In [2]:
#load train/test data
prob = FreeFlyer() #use default config, pass different config file oth.
config_fn = './free_flyer/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/free_flyer/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
# p_train, x_train, u_train, y_train, c_train, times_train = pickle.load(train_file)
train_data = pickle.load(train_file)
train_file.close()

test_file = open(dataset_fn+'/test.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
test_data = pickle.load(test_file)
test_file.close()

# test MLOPT solver

In [3]:
from solvers.mlopt import MLOPT

system = 'free_flyer'
prob_features = ['x0', 'obstacles']
mlopt_obj = MLOPT(system, prob, prob_features)

n_features = 36
mlopt_obj.construct_strategies(n_features, train_data)
print(mlopt_obj.n_strategies)

64988


In [ ]:
mlopt_obj.setup_network()

fn_saved = 'models/mlopt_free_flyer_20200624_1449.pt'
fn_saved = 'models/mlopt_free_flyer_20200629_1323.pt'
mlopt_obj.load_network(fn_saved)

mlopt_obj.model

In [ ]:
mlopt_obj.training_params['TRAINING_ITERATIONS'] = 100
mlopt_obj.train()
print(mlopt_obj.model_fn)

In [ ]:
prob_params = {}
idx = np.random.randint(test_data[1].shape[0])

p_test = test_data[0]
for k in p_test.keys():
    prob_params[k] = p_test[k][idx]
prob_success, cost, total_time, n_evals = mlopt_obj.forward(prob_params)

if prob_success:
    print(cost)
else:
    print('Failed')

In [ ]:
prob_success, true_cost, solve_time, optvals = mlopt_obj.problem.solve_micp(prob_params)

true_cost

In [ ]:
n_succ = 0
count = 0

for ii in range(100):
    idx = np.random.randint(train_data[1].shape[0])
    prob_params = {}
    p_test = train_data[0]
    for k in p_test.keys():
        prob_params[k] = p_test[k][idx]
    prob_success, cost, total_time, n_evals = mlopt_obj.forward(prob_params)

    count += 1
    if prob_success:
        n_succ += 1

print(float(n_succ)/ float(count))

In [ ]:
idx = np.random.randint(train_data[1].shape[0])
x0 = train_data[0]['x0'][idx]
n_obs = train_data[0]['obstacles'][idx].shape[1]
posmin, posmax = mlopt_obj.problem.posmin, mlopt_obj.problem.posmax

obstacles = []
for ii_obs in range(n_obs):
    obs = train_data[0]['obstacles'][idx][:,ii_obs]
    obstacles.append(obs)

if len(obstacles) is n_obs:
    plt.axes()
    for obstacle in obstacles:
        rectangle = plt.Rectangle((obstacle[0], obstacle[2]), \
                                  obstacle[1]-obstacle[0], obstacle[3]-obstacle[2], \
                                 fc='red', ec='blue')
        plt.gca().add_patch(rectangle)
        plt.axis('scaled')

    circle = plt.Circle((x0[0],x0[1]), 0.04, fc='blue',ec="green")
    plt.gca().add_patch(circle)    

    ax = plt.gca()
    ax.margins(0)
    ax.set(xlim=(posmin[0],posmax[0]), ylim=(posmin[1],posmax[1]))
    plt.show()

# test task-specific MLOPT

In [7]:
from solvers.mlopt_ff import MLOPT_FF

In [30]:
system = 'free_flyer'
prob_features = ['x0', 'obstacles']
mlopt_ts_obj = MLOPT_FF(system, prob, prob_features)

n_features = 36 + prob.n_obs
mlopt_ts_obj.construct_strategies(n_features, train_data)
print(mlopt_ts_obj.n_strategies)

> /home/acauligi/cs_234/project/mlopt-micp/solvers/mlopt_ff.py(98)construct_strategies()
-> if obs_strat not in self.strategy_dict.keys():


(Pdb)  obs_strrat


*** NameError: name 'obs_strrat' is not defined


(Pdb)  obs_strat


(1, 2, 3, 33, 34, 35, 65, 66, 67, 97, 98, 99, 129, 130, 131)


...

> /home/acauligi/cs_234/project/mlopt-micp/solvers/mlopt_ff.py(96)construct_strategies()
-> pdb.set_trace()


(Pdb)  obs_strat


(13, 14, 45, 46, 77, 78, 79, 109, 110, 111, 141, 142, 143)


(Pdb)  q


BdbQuit: 

In [34]:
prob = FreeFlyer() #use default config, pass different config file oth.
system = 'free_flyer'
prob_features = ['x0', 'obstacles']
mlopt_ts_obj = MLOPT_FF(system, prob, prob_features)

n_features = 36 + prob.n_obs
mlopt_ts_obj.construct_strategies(n_features, train_data)
print(mlopt_ts_obj.n_strategies)

979


In [36]:
mlopt_ts_obj.setup_network()

# mlopt_ts_obj.load_network(fn_saved)

mlopt_ts_obj.model

FFNet(
  (activation): ReLU()
  (layers): ModuleList(
    (0): Linear(in_features=44, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=979, bias=True)
  )
)

In [37]:
mlopt_ts_obj.training_params['TRAINING_ITERATIONS'] = 1000
mlopt_ts_obj.train()
print(mlopt_ts_obj.model_fn)

loss:   3.262643575668335,   acc:  0.3125
Done with epoch 0 in 2.3208892345428467s
loss:   3.04656982421875,   acc:  0.375
Done with epoch 1 in 2.3787901401519775s
loss:   3.608766794204712,   acc:  0.34375
loss:   3.9276769161224365,   acc:  0.125
Done with epoch 2 in 2.450430154800415s
loss:   2.361124277114868,   acc:  0.4375
Done with epoch 3 in 2.366983652114868s
loss:   3.0866901874542236,   acc:  0.4375
loss:   3.332869529724121,   acc:  0.28125
Done with epoch 4 in 2.3948206901550293s
loss:   2.893850326538086,   acc:  0.40625
Done with epoch 5 in 2.3331923484802246s
loss:   2.7058939933776855,   acc:  0.375
Done with epoch 6 in 2.357694149017334s
loss:   2.658712148666382,   acc:  0.5
loss:   2.716782569885254,   acc:  0.40625
Done with epoch 7 in 2.361222267150879s
loss:   2.936413049697876,   acc:  0.4375
Done with epoch 8 in 2.3057427406311035s
loss:   3.538048267364502,   acc:  0.1875
loss:   2.945460319519043,   acc:  0.375
Done with epoch 9 in 2.3763668537139893s
loss:  

KeyboardInterrupt: 

In [52]:
prob_params = {}
idx = np.random.randint(test_data[1].shape[0])

p_test = test_data[0]
for k in p_test.keys():
    prob_params[k] = p_test[k][idx]
prob_success, cost, total_time, n_evals = mlopt_ts_obj.forward(prob_params)

if prob_success:
    print(cost)
else:
    print('Failed')

86.93769079671627


In [55]:
n_succ = 0
count = 0
for ii in range(25):
    prob_params = {}
    idx = np.random.randint(test_data[1].shape[0])

    p_test = test_data[0]
    for k in p_test.keys():
        prob_params[k] = p_test[k][idx]
    prob_success, cost, total_time, n_evals = mlopt_ts_obj.forward(prob_params)

    count += 1
    if prob_success:
        n_succ += 1

float(n_succ) / float(count)

0.16

In [53]:
prob_success, true_cost, solve_time, optvals = mlopt_ts_obj.problem.solve_micp(prob_params)

true_cost

86.93769129480138

In [ ]:
x0 = train_data[0]['x0'][idx]
n_obs = mlopt_ts_obj.problem.n_obs
posmin, posmax = mlopt_ts_obj.problem.posmin, mlopt_ts_obj.problem.posmax

obstacles = []
for ii_obs in range(n_obs):
    obs = train_data[0]['obstacles'][idx][:,ii_obs]
    obstacles.append(obs)

X = optvals[0]

plt.axes()
for obstacle in obstacles:
    rectangle = plt.Rectangle((obstacle[0], obstacle[2]), \
                              obstacle[1]-obstacle[0], obstacle[3]-obstacle[2], \
                             fc='red', ec='blue')
    plt.gca().add_patch(rectangle)
    plt.axis('scaled')

for ii in range(mlopt_ts_obj.problem.N):
    circle = plt.Circle((X[0,ii],X[1,ii]), 0.04, fc='blue',ec="green")
    plt.gca().add_patch(circle)

ax = plt.gca()
ax.margins(0)
ax.set(xlim=(posmin[0],posmax[0]), ylim=(posmin[1],posmax[1]))
plt.show()

# test Regression solver

In [ ]:
from solvers.regression import Regression

system = 'free_flyer'
prob_features = ['x0', 'obstacles']
reg_obj = Regression(system, prob, prob_features)

n_features = 36
reg_obj.construct_strategies(n_features, train_data)
print(reg_obj.n_strategies)

In [ ]:
reg_obj.setup_network()

reg_obj.model

In [ ]:
reg_obj.training_params['TRAINING_ITERATIONS'] = 10000
reg_obj.training_params['CHECKPOINT_AFTER'] = 200000
reg_obj.train()
print(reg_obj.model_fn)

In [ ]:
prob_params = {}
idx = np.random.randint(test_data[1].shape[0])

p_test = test_data[0]
for k in p_test.keys():
    prob_params[k] = p_test[k][idx]
prob_params

In [ ]:
prob_success, cost, total_time = reg_obj.forward(prob_params)

if prob_success:
    print(cost)
else:
    print('Failed')

In [ ]:
n_succ = 0
for idx in range(100):
    prob_params = {}

    p_test = test_data[0]
    for k in p_test.keys():
        prob_params[k] = p_test[k][idx]

    prob_success, cost, total_time = reg_obj.forward(prob_params)
    if prob_success:
        n_succ +=1

print(float(n_succ) / 100)

In [ ]:
prob_success, true_cost, solve_time, optvals = reg_obj.problem.solve_micp(prob_params)

true_cost